# Cine21 리뷰 키워드 추출

## 1. Import libraries & set options

In [34]:
import pandas as pd
import numpy as np
import time

pd.set_option('display.max_rows', 1000)

## 2. Prepare raw data

In [2]:
raw_df = pd.read_csv("./cine21_final_raw_df.csv")
raw_df

,title,release_date,genre,rating,story,review,url
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",penny2003\n2019-06-02 23:31:32\n5\n90년대 중반 심은하...,http://www.cine21.com/movie/info/?movie_id=52988
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",hwachul92\n2019-12-28 18:46:00\n2\n전개도 너무 시시한데...,http://www.cine21.com/movie/info/?movie_id=55539
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",penny2003\n2019-09-22 20:59:10\n6\n뻔한 재미와 감동이지...,http://www.cine21.com/movie/info/?movie_id=55031
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,"hwachul92\n2019-04-27 11:03:41\n7\n제2차 세계대전 때,...",http://www.cine21.com/movie/info/?movie_id=54607
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",There is no reviews for this movie,http://www.cine21.com/movie/info/?movie_id=54778
...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,Unable to get rating,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,penny2003\n2021-02-22 21:25:51\n4\n재난 스릴러물은데 전...,http://www.cine21.com/movie/info/?movie_id=57226
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",penny2003\n2021-08-01 16:45:18\n6\n맨 마지막 장면을 위...,http://www.cine21.com/movie/info/?movie_id=57001
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",filmone1\n2021-08-01 06:46:36\n5\n<호스트:접속금지>는 ...,http://www.cine21.com/movie/info/?movie_id=57001
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",thrill5\n2021-05-23 00:37:40\n10\n시대가 좋아졌다고는 하...,http://www.cine21.com/movie/info/?movie_id=57892


In [4]:
# raw_df 대신 전처리에 사용할 복사본 df: tmp 생성
tmp = raw_df
tmp

,title,release_date,genre,rating,story,review,url
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",penny2003\n2019-06-02 23:31:32\n5\n90년대 중반 심은하...,http://www.cine21.com/movie/info/?movie_id=52988
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",hwachul92\n2019-12-28 18:46:00\n2\n전개도 너무 시시한데...,http://www.cine21.com/movie/info/?movie_id=55539
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",penny2003\n2019-09-22 20:59:10\n6\n뻔한 재미와 감동이지...,http://www.cine21.com/movie/info/?movie_id=55031
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,"hwachul92\n2019-04-27 11:03:41\n7\n제2차 세계대전 때,...",http://www.cine21.com/movie/info/?movie_id=54607
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",There is no reviews for this movie,http://www.cine21.com/movie/info/?movie_id=54778
...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,Unable to get rating,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,penny2003\n2021-02-22 21:25:51\n4\n재난 스릴러물은데 전...,http://www.cine21.com/movie/info/?movie_id=57226
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",penny2003\n2021-08-01 16:45:18\n6\n맨 마지막 장면을 위...,http://www.cine21.com/movie/info/?movie_id=57001
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",filmone1\n2021-08-01 06:46:36\n5\n<호스트:접속금지>는 ...,http://www.cine21.com/movie/info/?movie_id=57001
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",thrill5\n2021-05-23 00:37:40\n10\n시대가 좋아졌다고는 하...,http://www.cine21.com/movie/info/?movie_id=57892


### 씨네21에서 수집한 데이터 살펴보기

In [5]:
# 전체 영화 수
raw_df['title'].nunique() # 1127

1127

In [6]:
# 리뷰가 단 한 건이라도 있는 영화의 수
review_available_df = raw_df[raw_df.review != 'There is no reviews for this movie']
review_available_df.title.nunique() # 495

495

In [7]:
# 리뷰가 단 한 건도 없는 영화의 수
no_review_df = raw_df[raw_df.review == 'There is no reviews for this movie']
no_review_df.title.nunique() # 632

632

### 2. 1. 유저 아이디 삭제

In [8]:
# The expand parameter is False and that is why a series with List of strings is returned instead of a data frame.
review = tmp["review"].str.split("\n", n = 1, expand = True)
review

,0,1
0,penny2003,2019-06-02 23:31:32\n5\n90년대 중반 심은하 주연 납량 특집드라...
1,hwachul92,2019-12-28 18:46:00\n2\n전개도 너무 시시한데다 결말도 엄청 이상...
2,penny2003,2019-09-22 20:59:10\n6\n뻔한 재미와 감동이지만 시츄에이션 설정이...
3,hwachul92,"2019-04-27 11:03:41\n7\n제2차 세계대전 때, 나치로부터 노르웨이..."
4,There is no reviews for this movie,None
...,...,...
1806,penny2003,2021-02-22 21:25:51\n4\n재난 스릴러물은데 전혀 긴장감이 안 느껴...
1807,penny2003,2021-08-01 16:45:18\n6\n맨 마지막 장면을 위한 빌드업이 조금 루...
1808,filmone1,2021-08-01 06:46:36\n5\n<호스트:접속금지>는 코로나 시대에 자가...
1809,thrill5,2021-05-23 00:37:40\n10\n시대가 좋아졌다고는 하지만 좋아진 시대...


In [9]:
tmp['review'] = review[1].reset_index(drop=True)
tmp

,title,release_date,genre,rating,story,review,url
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",2019-06-02 23:31:32\n5\n90년대 중반 심은하 주연 납량 특집드라...,http://www.cine21.com/movie/info/?movie_id=52988
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",2019-12-28 18:46:00\n2\n전개도 너무 시시한데다 결말도 엄청 이상...,http://www.cine21.com/movie/info/?movie_id=55539
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",2019-09-22 20:59:10\n6\n뻔한 재미와 감동이지만 시츄에이션 설정이...,http://www.cine21.com/movie/info/?movie_id=55031
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,"2019-04-27 11:03:41\n7\n제2차 세계대전 때, 나치로부터 노르웨이...",http://www.cine21.com/movie/info/?movie_id=54607
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",None,http://www.cine21.com/movie/info/?movie_id=54778
...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,Unable to get rating,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,2021-02-22 21:25:51\n4\n재난 스릴러물은데 전혀 긴장감이 안 느껴...,http://www.cine21.com/movie/info/?movie_id=57226
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",2021-08-01 16:45:18\n6\n맨 마지막 장면을 위한 빌드업이 조금 루...,http://www.cine21.com/movie/info/?movie_id=57001
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",2021-08-01 06:46:36\n5\n<호스트:접속금지>는 코로나 시대에 자가...,http://www.cine21.com/movie/info/?movie_id=57001
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",2021-05-23 00:37:40\n10\n시대가 좋아졌다고는 하지만 좋아진 시대...,http://www.cine21.com/movie/info/?movie_id=57892


### 2. 2. 작성일: 시간 삭제

In [10]:
review = tmp["review"].str.split("\n", n = 1, expand = True)
review

,0,1
0,2019-06-02 23:31:32,5\n90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다...
1,2019-12-28 18:46:00,2\n전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가...
2,2019-09-22 20:59:10,6\n뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.\n신고
3,2019-04-27 11:03:41,"7\n제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 ..."
4,None,None
...,...,...
1806,2021-02-22 21:25:51,4\n재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제\n신고
1807,2021-08-01 16:45:18,6\n맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.\n신고
1808,2021-08-01 06:46:36,5\n<호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 ...
1809,2021-05-23 00:37:40,10\n시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며\n\n좋...


In [11]:
tmp['write_date'] = review[0].reset_index(drop=True)
tmp['review'] = review[1].reset_index(drop=True)
tmp

,title,release_date,genre,rating,story,review,url,write_date
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",5\n90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다...,http://www.cine21.com/movie/info/?movie_id=52988,2019-06-02 23:31:32
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",2\n전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가...,http://www.cine21.com/movie/info/?movie_id=55539,2019-12-28 18:46:00
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",6\n뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.\n신고,http://www.cine21.com/movie/info/?movie_id=55031,2019-09-22 20:59:10
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,"7\n제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 ...",http://www.cine21.com/movie/info/?movie_id=54607,2019-04-27 11:03:41
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",None,http://www.cine21.com/movie/info/?movie_id=54778,None
...,...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,Unable to get rating,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,4\n재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제\n신고,http://www.cine21.com/movie/info/?movie_id=57226,2021-02-22 21:25:51
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",6\n맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.\n신고,http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01 16:45:18
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",5\n<호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 ...,http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01 06:46:36
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",10\n시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며\n\n좋...,http://www.cine21.com/movie/info/?movie_id=57892,2021-05-23 00:37:40


### 2. 3. score 컬럼 추가

In [12]:
review = tmp["review"].str.split("\n", n = 1, expand = True)
review

,0,1
0,5,90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다.\n신고
1,2,전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가는지 ...
2,6,뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.\n신고
3,7,"제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 비밀 ..."
4,None,None
...,...,...
1806,4,재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제\n신고
1807,6,맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.\n신고
1808,5,<호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 프로그...
1809,10,시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며\n\n좋든 싫든...


In [13]:
tmp['score'] = review[0].reset_index(drop=True)
tmp['review'] = review[1].reset_index(drop=True)
tmp

,title,release_date,genre,rating,story,review,url,write_date,score
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다.\n신고,http://www.cine21.com/movie/info/?movie_id=52988,2019-06-02 23:31:32,5
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가는지 ...,http://www.cine21.com/movie/info/?movie_id=55539,2019-12-28 18:46:00,2
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.\n신고,http://www.cine21.com/movie/info/?movie_id=55031,2019-09-22 20:59:10,6
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,"제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 비밀 ...",http://www.cine21.com/movie/info/?movie_id=54607,2019-04-27 11:03:41,7
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",None,http://www.cine21.com/movie/info/?movie_id=54778,None,None
...,...,...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,Unable to get rating,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제\n신고,http://www.cine21.com/movie/info/?movie_id=57226,2021-02-22 21:25:51,4
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.\n신고,http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01 16:45:18,6
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",<호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 프로그...,http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01 06:46:36,5
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며\n\n좋든 싫든...,http://www.cine21.com/movie/info/?movie_id=57892,2021-05-23 00:37:40,10


### 2. 4. '\n' replace & '\n신고' & '\nmore' 삭제

In [14]:
review = tmp['review'].str.replace('\n신고', '').replace('\nmore', '')
review

0            90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다.
1       전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가는지 ...
2                          뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.
3       제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 비밀 ...
4                                                    None
                              ...                        
1806                         재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제
1807              맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.
1808    <호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 프로그...
1809    시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며\n\n좋든 싫든...
1810                          혼자 살고 비슷한 성향이라 많이 공감이 가는 영화
Name: review, Length: 1811, dtype: object

In [15]:
tmp['review'] = review
tmp

,title,release_date,genre,rating,story,review,url,write_date,score
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다.,http://www.cine21.com/movie/info/?movie_id=52988,2019-06-02 23:31:32,5
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가는지 ...,http://www.cine21.com/movie/info/?movie_id=55539,2019-12-28 18:46:00,2
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.,http://www.cine21.com/movie/info/?movie_id=55031,2019-09-22 20:59:10,6
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,"제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 비밀 ...",http://www.cine21.com/movie/info/?movie_id=54607,2019-04-27 11:03:41,7
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",None,http://www.cine21.com/movie/info/?movie_id=54778,None,None
...,...,...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,Unable to get rating,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제,http://www.cine21.com/movie/info/?movie_id=57226,2021-02-22 21:25:51,4
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.,http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01 16:45:18,6
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",<호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 프로그...,http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01 06:46:36,5
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며\n\n좋든 싫든...,http://www.cine21.com/movie/info/?movie_id=57892,2021-05-23 00:37:40,10


### 2.5. review 컬럼에서 \n를 삭제

In [16]:
review = tmp['review'].str.replace('\n', ' ')
review

0            90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다.
1       전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가는지 ...
2                          뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.
3       제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 비밀 ...
4                                                    None
                              ...                        
1806                         재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제
1807              맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.
1808    <호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 프로그...
1809    시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며  좋든 싫든 인...
1810                          혼자 살고 비슷한 성향이라 많이 공감이 가는 영화
Name: review, Length: 1811, dtype: object

In [17]:
tmp['review'] = review
tmp

,title,release_date,genre,rating,story,review,url,write_date,score
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다.,http://www.cine21.com/movie/info/?movie_id=52988,2019-06-02 23:31:32,5
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가는지 ...,http://www.cine21.com/movie/info/?movie_id=55539,2019-12-28 18:46:00,2
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.,http://www.cine21.com/movie/info/?movie_id=55031,2019-09-22 20:59:10,6
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,"제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 비밀 ...",http://www.cine21.com/movie/info/?movie_id=54607,2019-04-27 11:03:41,7
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",None,http://www.cine21.com/movie/info/?movie_id=54778,None,None
...,...,...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,Unable to get rating,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제,http://www.cine21.com/movie/info/?movie_id=57226,2021-02-22 21:25:51,4
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.,http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01 16:45:18,6
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",<호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 프로그...,http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01 06:46:36,5
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며 좋든 싫든 인...,http://www.cine21.com/movie/info/?movie_id=57892,2021-05-23 00:37:40,10


### 2. 6. write_date 컬럼에서 작성 시간-분-초 삭제

In [18]:
write_time = tmp["write_date"].str.split(" ", n = 1, expand = True)
write_time

,0,1
0,2019-06-02,23:31:32
1,2019-12-28,18:46:00
2,2019-09-22,20:59:10
3,2019-04-27,11:03:41
4,None,None
...,...,...
1806,2021-02-22,21:25:51
1807,2021-08-01,16:45:18
1808,2021-08-01,06:46:36
1809,2021-05-23,00:37:40


In [19]:
tmp['write_date'] = write_time[0]
tmp

,title,release_date,genre,rating,story,review,url,write_date,score
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다.,http://www.cine21.com/movie/info/?movie_id=52988,2019-06-02,5
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가는지 ...,http://www.cine21.com/movie/info/?movie_id=55539,2019-12-28,2
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.,http://www.cine21.com/movie/info/?movie_id=55031,2019-09-22,6
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,"제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 비밀 ...",http://www.cine21.com/movie/info/?movie_id=54607,2019-04-27,7
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",None,http://www.cine21.com/movie/info/?movie_id=54778,None,None
...,...,...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,Unable to get rating,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제,http://www.cine21.com/movie/info/?movie_id=57226,2021-02-22,4
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.,http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01,6
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,Unable to get rating,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",<호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 프로그...,http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01,5
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며 좋든 싫든 인...,http://www.cine21.com/movie/info/?movie_id=57892,2021-05-23,10


### 2. 7. rating 컬럼 결측값 채우기

In [20]:
# 등급 정보 없는 영화 815편
tmp[tmp.rating == 'Unable to get rating'].title.nunique()

# None 값으로 채우기
tmp['rating'] = tmp['rating'].replace("Unable to get rating", None)

# 다시 확인
tmp[tmp.rating == 'Unable to get rating'].title.nunique() # 3건... 앞에 세 건은 왜 안 바뀌는 거야 ㅠㅠ

3

### 2. 8. story 컬럼 결측값 채우기

In [21]:
tmp['story'] = tmp['story'].replace("Unable to get story", None)
tmp[tmp.story == 'Unable to get story'].title.nunique()

0

### 2. 9. 컬럼 자리 바꾸기

In [22]:
# tmp 컬럼 자리 옮기기
cine21_final_df = tmp[['title', 'release_date', 'genre', 'rating', 'story', 'url', 'write_date', 'score', 'review']]
cine21_final_df

,title,release_date,genre,rating,story,url,write_date,score,review
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",http://www.cine21.com/movie/info/?movie_id=52988,2019-06-02,5,90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다.
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",http://www.cine21.com/movie/info/?movie_id=55539,2019-12-28,2,전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가는지 ...
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",http://www.cine21.com/movie/info/?movie_id=55031,2019-09-22,6,뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,http://www.cine21.com/movie/info/?movie_id=54607,2019-04-27,7,"제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 비밀 ..."
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",http://www.cine21.com/movie/info/?movie_id=54778,None,None,None
...,...,...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,12세이상관람가,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,http://www.cine21.com/movie/info/?movie_id=57226,2021-02-22,4,재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,12세이상관람가,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01,6,맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,12세이상관람가,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01,5,<호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 프로그...
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",http://www.cine21.com/movie/info/?movie_id=57892,2021-05-23,10,시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며 좋든 싫든 인...


In [23]:
cine21_final_df

,title,release_date,genre,rating,story,url,write_date,score,review
0,0.0MHz,개봉일 : 2019-05-29,공포,Unable to get rating,"에디슨의 마지막 발명품, 유령 탐지기?“증명할 수 없는 미스터리란 없다”가지 말라는...",http://www.cine21.com/movie/info/?movie_id=52988,2019-06-02,5,90년대 중반 심은하 주연 납량 특집드라마 'M' 을 다시 보는 느낌이 들었다.
1,10 미니츠 곤,개봉일 : 2019-11,"범죄, 액션",Unable to get rating,"룰이 깨진 순간, 모든 멤버가 타겟이 된다!동물적인 직감과 1초의 오차도 용납하지 ...",http://www.cine21.com/movie/info/?movie_id=55539,2019-12-28,2,전개도 너무 시시한데다 결말도 엄청 이상하게 지어서 내용이 뭐가 어떻게 돌아가는지 ...
2,100일 동안 100가지로 100퍼센트 행복찾기,개봉일 : 2019-09-12,코미디,Unable to get rating,"어렸을 적부터 함께 자라고, IT 회사도 공동 운영,사는 집도 아랫집, 윗집으로 이...",http://www.cine21.com/movie/info/?movie_id=55031,2019-09-22,6,뻔한 재미와 감동이지만 시츄에이션 설정이 독창적이었다.
3,12번째 솔저,개봉일 : 2019-04-11,전쟁,12세이상관람가,제2차 세계대전 나치에 점령된 노르웨이.나치를 함락시킬 유일한 ‘마틴 레드 작전’에...,http://www.cine21.com/movie/info/?movie_id=54607,2019-04-27,7,"제2차 세계대전 때, 나치로부터 노르웨이를 되찾기 위한 작전을 펼친 노르웨이 비밀 ..."
4,13년의 공백,개봉일 : 2019-07-04,"드라마, 가족",12세이상관람가,"“아버지가 너무 싫습니다.그런데, 조금은 좋아하는 것 같기도 합니다.”담배를 사러 ...",http://www.cine21.com/movie/info/?movie_id=54778,None,None,None
...,...,...,...,...,...,...,...,...,...
1806,호라이즌 라인,개봉일 : 2021-02-10,스릴러,12세이상관람가,계기판도 GPS도 조종사도 없다!내 안의 생존 본능을 깨워라!에메랄드 빛 해안을 낀...,http://www.cine21.com/movie/info/?movie_id=57226,2021-02-22,4,재난 스릴러물은데 전혀 긴장감이 안 느껴지는게 문제
1807,호스트: 접속금지,개봉일 : 2021-07-21,공포,12세이상관람가,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01,6,맨 마지막 장면을 위한 빌드업이 조금 루즈했지만 마지막은 진짜 놀랐다.
1808,호스트: 접속금지,개봉일 : 2021-07-21,공포,12세이상관람가,"전 세계가 극찬한 100% 리얼 팬데믹 호러!지금, 당신의 랜선미팅에 무언가가 접속...",http://www.cine21.com/movie/info/?movie_id=57001,2021-08-01,5,<호스트:접속금지>는 코로나 시대에 자가 격리 된 여섯 명의 친구가 화상채팅 프로그...
1809,혼자 사는 사람들,개봉일 : 2021-05-19,드라마,12세이상관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.사람들은 자꾸 말을 걸어오지만, 진아는 그저...",http://www.cine21.com/movie/info/?movie_id=57892,2021-05-23,10,시대가 좋아졌다고는 하지만 좋아진 시대를 누리는 이들은 많지 않으며 좋든 싫든 인...


### 2. 10. 전처리 끝난 씨네21 리뷰 데이터를 csv 파일로 저장

In [24]:
cine21_final_df.to_csv("cine21_final_df.csv", index=False, encoding='utf-8-sig')